# Data exploration 

---

Group name: O

---


## Introduction

*This section includes a short description of the data* 

## Setup

In [201]:
import pandas as pd
import altair as alt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

## Data

## Import data

In [202]:
#DATAGRADSTUDENTS = "https://raw.githubusercontent.com/FabioRaab/bigData-HW1/main/data/external/grad-students.csv"

In [203]:
PoliceKillingsData = "https://raw.githubusercontent.com/FabioRaab/bigData-HW1/main/data/external/police_killings.csv"

### Data structure

In [204]:
df = pd.read_csv(PoliceKillingsData)
print(df)

df.info()

                   name age  gender    raceethnicity     month  day  year  \
0    A'donte Washington  16    Male            Black  February   23  2015   
1        Aaron Rutledge  27    Male            White     April    2  2015   
2           Aaron Siler  26    Male            White     March   14  2015   
3          Aaron Valdez  25    Male  Hispanic/Latino     March   11  2015   
4          Adam Jovicic  29    Male            White     March   19  2015   
..                  ...  ..     ...              ...       ...  ...   ...   
462  William Chapman II  18    Male            Black     April   22  2015   
463    William Dick III  28    Male  Native American     April    4  2015   
464       William Poole  52    Male            White     March   16  2015   
465   Yuvette Henderson  38  Female            Black  February    3  2015   
466       Zaki Shinwary  48    Male          Unknown   January   16  2015   

                                 streetaddress          city state  ...  \


### Data corrections

In [205]:
df['cause'] = df['cause'].astype("category")

In [206]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   name                  467 non-null    object  
 1   age                   467 non-null    object  
 2   gender                467 non-null    object  
 3   raceethnicity         467 non-null    object  
 4   month                 467 non-null    object  
 5   day                   467 non-null    int64   
 6   year                  467 non-null    int64   
 7   streetaddress         463 non-null    object  
 8   city                  467 non-null    object  
 9   state                 467 non-null    object  
 10  latitude              467 non-null    float64 
 11  longitude             467 non-null    float64 
 12  state_fp              467 non-null    int64   
 13  county_fp             467 non-null    int64   
 14  tract_ce              467 non-null    int64   
 15  geo_id

### Variable lists

In [218]:
# make a list of variables you want to use
var_list = ['gender', 'cause']

In [219]:
source = df[var_list]

In [220]:
source

,gender,cause
0,Male,Gunshot
1,Male,Gunshot
2,Male,Gunshot
3,Male,Gunshot
4,Male,Gunshot
...,...,...
462,Male,Gunshot
463,Male,Taser
464,Male,Gunshot
465,Female,Gunshot


# Visualization 1: Bar plot analyzing cause of death

I want to analyse the main causes of death accordig to the data set. In order to do so, I will be visualizing all causes of death in a bar chart since this is the easiest way to see, which cause of death is the most frequent one. We'll also make this bar plot interactive to enable the viewer to see the exact count of records.

In [221]:
ChartCause = alt.Chart(source).mark_bar().encode(
     x=alt.X('cause',
        sort='-y' ),
    y=alt.Y('count(cause)')
)

In [229]:
ChartCause = alt.Chart(source).mark_bar().encode(
    x=alt.X('cause', 
            sort= '-y',
            #für die Achseneinstellung benutzen wir axis
            axis=alt.Axis(title="Cause of death", # title of x axis: Cause of death
                          labelAngle=0)), # angle of x axis text: 0
      y=alt.Y('count(cause)'),
      tooltip=['count(cause)'] 

).interactive(

).properties( 
    title= 'Cause of death',
    width= 500,
    height= 400
)
ChartCause

alt.Chart(...)

# Visualization 2: Pie chart analysing race/ethnicity of deceased

Next, I want to analyse the distrubution of race/ethincity of the deceased people of the data set to find out why race/ethinicity is most likely to be targeted.

In [233]:
df.raceethnicity = df.raceethnicity.astype("category")

# create data for pie chart
source = pd.DataFrame(df.raceethnicity.value_counts())

# set index to column
source = source.reset_index()

# rename columns
source.rename(columns={"index": "race", "raceethnicity": "value"}, inplace=True)

source

,race,value
0,White,236
1,Black,135
2,Hispanic/Latino,67
3,Unknown,15
4,Asian/Pacific Islander,10
5,Native American,4


In [234]:
chartRace = alt.Chart(source).mark_arc().encode(
    theta=alt.Theta(field="value", type="quantitative"),
    color=alt.Color(field="race", type="nominal"),
)

pie = chartRace.mark_arc(outerRadius=125)

pie

alt.Chart(...)

# Visualization 3: Map

In [214]:
df.state = df.state.astype("category")

# Visualization 4: Year of deseased's death